## Import libraries

In [1]:
import numpy as np
import pandas as pd


---------------------------------------------------------------------------------------------

# 1st STEP: SLICING SUPERSET

## Reading two dasets with 227 countries which repeat ciclycally for 5 different attributes:
*We have 5 datasets inside of one 'SUPER'dataset*. *Need to split them up in five separately*
### total
### nuclear
### renewable
### fossil fuel
### hydroelectric pumped storage
---------------------------------------------------------------------------------------------


In [2]:
capacity = pd.read_csv('../raw_data/electricity_capacity.csv',sep=',',header=4, quotechar='"',decimal=',')
generation = pd.read_csv('../raw_data/electricity_generation.csv',sep=',',header=4, quotechar='"',decimal=',')


## For each dataset we define the names we want to split it up: their name categories

In [3]:
categories_generation = ['Generation', 'Nuclear', 'Renewables', 'Fossil Fuels', 'Hydroelectric Pumped Storage']
categories_capacity = ['Capacity', 'Nuclear', 'Renewables', 'Fossil Fuels', 'Hydroelectric Pumped Storage']

## We save the index where the different datasets have to been splitted up

In [4]:
indexes = []
for i in categories_generation:
    indexes.append(generation[generation['Unnamed: 1'] == i].index[0])

## Slicing every dataset (from a total of 2: generation and capacity of eletrical energy) and save them in two dataframes.

In [5]:
#definition of slicing function

def slicing(df, indexes):
    dataframes = []
    for i in range(len(indexes) - 1):
        dataframes.append(df.iloc[indexes[i]: indexes[i + 1]])
    else:
        dataframes.append(df.iloc[indexes[i + 1]:])
    return dataframes

##saving the datases in a list of dataframes

generation = slicing(generation,indexes)
capacity = slicing (capacity,indexes)

### Rename all dataset (5 datasets from the 2 initial 'SUPER'dataset), makes a total of 10

In [6]:
gen_total= generation[0].reset_index(drop=True)
gen_nuclear = generation[1].reset_index(drop=True)
gen_renewable = generation[2].reset_index(drop=True) 
gen_fossil = generation[3].reset_index(drop=True) 
gen_hidro = generation[4].reset_index(drop=True) 

cap_total = capacity[0].reset_index(drop=True)
cap_nuclear = capacity[1].reset_index(drop=True)
cap_renewable = capacity[2].reset_index(drop=True)
cap_fossil = capacity[3].reset_index(drop=True)
cap_hidro = capacity[4].reset_index(drop=True)




### Read the other 4 'simple' dataset

In [7]:
consumption = pd.read_csv('../raw_data/electricity_consumption.csv',sep=',',header=4, quotechar='"',decimal=',')
exports = pd.read_csv('../raw_data/electricity_exports.csv',sep=',',header=4, quotechar='"',decimal=',')
imports = pd.read_csv('../raw_data/electricity_imports.csv',sep=',',header=4, quotechar='"',decimal=',')
population = pd.read_csv('../raw_data/population.csv',sep=',',header=4, quotechar='"',decimal=',')

## Gathering all useful datasets as function of how many initial rows we need to drop, because there is just the category of attribute. 

## That information it in the name of the proper dataframe

In [8]:
features_01 = [consumption, exports, imports, population, gen_total,cap_total,gen_renewable,cap_renewable]
features_1 = [gen_nuclear, gen_fossil, gen_hidro, cap_nuclear,cap_fossil,cap_hidro]

In [9]:
features_01[2].head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,1980,1981,1982,1983,1984,1985,1986,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,NaN,Electricity,billion Kwh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Imports,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Afghanistan,billion Kwh,0,0,0,0,0,0,0,...,0.752,1.155,1.572,2.246,3.071,3.615,3.71,3.779,4.4,4.611
3,NaN,Albania,billion Kwh,0,0,0,0,0,0,0.005,...,2.434,1.965,1.005,3.475,3.23,2.323,3.25,2.355,1.827,2.915
4,NaN,Algeria,billion Kwh,0.07,0.103,0.09,0.109,0.167,0.121,0.145,...,0.274,0.33,0.736,0.657,0.936,0.295,0.686,0.61,0.257,0.537


---------------------------------------------------------------------------------------------

# 2nd STEP: CLEANING SUPERSET

## Define two functions: *CLEAN1* and *CLEAN2* to clean all datasets on an authomatized way.

---------------------------------------------------------------------------------------------


## CLEAN1 FUNCTION: *dataset with two first rows to drop*

### #1 dropping the empty  and the units columns

### #2 dropping the first 2 rows because there are the quantity to analyze and the category

### #3 rename the country column

### #4 Set'country' index

### #5 Replacing '--', '-' for nan. I discover for trial and error.

### #6 Drop the missing values.

### #7 Convert the data from objetct to number (float or int)

### #8  Transpose the matrix to have the country as index and year as columns.


In [10]:
def clean1(df):
    #1
    df.drop('Unnamed: 0',axis=1,inplace=True)

    df.drop('Unnamed: 2',axis=1,inplace=True)

    #2
    df.drop([0,1],axis=0,inplace=True)

    #3
    df=df.transpose()

    #4        
    header = df.iloc[0]
    df =df[1:]
    df.rename(columns =header,inplace=True)
    df.insert(0,'year', df.index)
    df.reset_index(drop =True, inplace=True)

    #5
    df.replace('--', np.nan, inplace=True)
    df.replace('-', np.nan, inplace=True)

    #6
    df.dropna(axis=1,inplace=True)

    #7
    df = df.apply(pd.to_numeric)
    return df


In [11]:
features_new = []
for df in features_01:
    features_new.append(clean1(df))

In [12]:
features_new[5]

,year,Afghanistan,Albania,Algeria,American Samoa,Angola,Antarctica,Antigua and Barbuda,Argentina,Australia,...,United States,Uruguay,Vanuatu,Venezuela,Vietnam,Wake Island,Western Sahara,Yemen,Zambia,Zimbabwe
0,1980,0.37400,0.5950,2.306,0.030,0.600,0.000,0.0240,12.909,24.1420,...,577.6140,0.944,0.0100,9.225,1.170,0,0.056,0.2340,1.6070,1.120
1,1981,0.42700,0.6200,2.809,0.032,0.600,0.000,0.0260,13.445,25.9380,...,597.3130,1.364,0.0100,10.125,1.180,0,0.056,0.2340,1.6070,1.120
2,1982,0.42700,0.6400,3.187,0.029,0.600,0.000,0.0260,14.163,28.0160,...,612.5930,1.339,0.0100,10.875,1.190,0,0.056,0.2540,1.6070,1.120
3,1983,0.45000,0.6900,3.436,0.025,0.600,0.000,0.0260,15.208,28.6350,...,619.6750,1.301,0.0110,12.499,1.220,0,0.056,0.2540,1.6070,1.120
4,1984,0.45000,0.7100,3.546,0.025,0.600,0.000,0.0260,15.975,31.2300,...,633.5310,1.291,0.0110,15.511,1.450,0,0.056,0.4200,1.5980,1.539
5,1985,0.47000,0.7350,3.736,0.031,0.600,0.000,0.0260,16.172,32.6950,...,653.2520,1.449,0.0110,18.194,1.550,0,0.056,0.6380,1.5980,1.539
6,1986,0.47000,0.7600,3.836,0.031,0.605,0.000,0.0260,16.510,33.3490,...,662.8790,1.449,0.0110,19.357,1.650,0,0.056,0.6380,1.5980,1.539
7,1987,0.48900,0.7670,3.936,0.031,0.617,0.000,0.0260,16.517,34.5930,...,672.1250,1.449,0.0110,17.444,1.750,0,0.056,0.6380,1.5830,1.634
8,1988,0.48900,0.7770,4.656,0.033,0.617,0.000,0.0260,16.972,34.9390,...,675.6050,1.681,0.0110,18.520,1.850,0,0.056,0.6380,1.5830,1.634
9,1989,0.49400,0.7770,4.657,0.033,0.617,0.000,0.0260,17.095,35.1650,...,709.7580,1.681,0.0110,18.520,1.950,0,0.056,0.6380,1.7000,1.634



## CLEAN2 FUNCTION: *dataset with just first row to drop*

### #1 dropping the empty  and the units columns

### #2 dropping the first 2 rows because there are the quantity to analyze and the category

### #3 rename the country column

### #4 Set'country' index

### #5 Replacing '--', '-' for nan. I discover for trial and error.

### #6 Drop the missing values.

### #7 Convert the data from objetct to number (float or int)

### #8  Transpose the matrix to have the country as index and year as columns.


In [13]:
def clean2(df):
    #1
    df.drop('Unnamed: 0',axis=1,inplace=True)

    df.drop('Unnamed: 2',axis=1,inplace=True)

    #2
    df.drop([0],axis=0,inplace=True)

    #3
    df=df.transpose()

    #4        
    header = df.iloc[0]
    df =df[1:]
    df.rename(columns =header,inplace=True)
    df.insert(0,'year', df.index)
    df.reset_index(drop =True, inplace=True)

    #5
    df.replace('--', np.nan, inplace=True)
    df.replace('-', np.nan, inplace=True)

    #6
    df.dropna(axis=1,inplace=True)

    #7
    df = df.apply(pd.to_numeric)
    return df


In [14]:
features_new1 = []
for df in features_1:
    features_new1.append(clean2(df))

In [15]:
features_new1[4]

,year,Afghanistan,Albania,Algeria,American Samoa,Angola,Antarctica,Antigua and Barbuda,Argentina,Australia,...,United States,Uruguay,Vanuatu,Venezuela,Vietnam,Wake Island,Western Sahara,Yemen,Zambia,Zimbabwe
0,1980,0.112,0.095,2.021,0.030,0.200,0,0.024,8.389,18.039,...,444.1040,0.483,0.010,6.543,0.890,0,0.056,0.234,0.069,0.487
1,1981,0.168,0.095,2.524,0.032,0.200,0,0.026,8.458,19.685,...,458.8710,0.483,0.010,7.443,0.900,0,0.056,0.234,0.069,0.487
2,1982,0.168,0.090,2.902,0.029,0.200,0,0.026,8.989,21.683,...,469.5580,0.458,0.010,7.075,0.910,0,0.056,0.254,0.069,0.487
3,1983,0.178,0.090,3.151,0.025,0.200,0,0.026,8.894,22.302,...,472.7660,0.420,0.011,8.073,0.920,0,0.056,0.254,0.069,0.487
4,1984,0.178,0.085,3.261,0.025,0.200,0,0.026,9.050,24.567,...,478.5790,0.410,0.011,10.546,1.000,0,0.056,0.420,0.060,0.906
5,1985,0.189,0.085,3.451,0.031,0.200,0,0.026,9.021,25.782,...,484.9550,0.410,0.011,9.646,1.080,0,0.056,0.638,0.060,0.906
6,1986,0.189,0.085,3.551,0.031,0.205,0,0.026,8.984,26.200,...,488.3380,0.410,0.011,9.451,1.160,0,0.056,0.638,0.060,0.906
7,1987,0.199,0.087,3.651,0.031,0.205,0,0.026,8.988,27.329,...,488.8420,0.410,0.011,8.675,1.240,0,0.056,0.638,0.045,1.001
8,1988,0.199,0.087,4.371,0.033,0.205,0,0.026,9.542,27.558,...,490.6100,0.485,0.011,8.520,1.320,0,0.056,0.638,0.045,1.001
9,1989,0.202,0.087,4.371,0.033,0.205,0,0.026,9.542,27.784,...,519.3920,0.485,0.011,8.520,1.400,0,0.056,0.638,0.030,1.001


---------------------------------------------------------------------------------------------

# 3rd Read and clean the ranking datasets
---------------------------------------------------------------------------------------------

In [16]:

ranking_consumption_2017 = pd.read_csv('../raw_data/ranking/electricity_net_consumption_2017.csv',sep=',',header=5, quotechar='"',decimal=',',index_col='Ranking')
ranking_renewable_2017 = pd.read_csv('../raw_data/ranking/renewable_electricity_net_generation_2017.csv',sep=',',header=5, quotechar='"',decimal=',',index_col='Ranking')
ranking_norenewable_2017 = pd.read_csv('../raw_data/ranking/fossil_fuels_electricity_net_generation_2017.csv',sep=',',header=5, quotechar='"',decimal=',',index_col='Ranking')
ranking_CO2_emissions_2016 = pd.read_csv('../raw_data/ranking/CO2_emissions_2016.csv',sep=',',header=5, quotechar='"',decimal=',',index_col='Ranking')
rankings=list([ranking_consumption_2017, ranking_renewable_2017, ranking_norenewable_2017,ranking_CO2_emissions_2016])

In [17]:

for item in rankings:
            item.drop('Unnamed: 0',axis=1, inplace=True)
            item['Value'] = pd.to_numeric(item['Value'],errors='coerce')
            item.dropna(inplace=True)
            

In [18]:

#datasets
features_new[0].to_csv('../cleaned_data/consumption.csv',index= False)
features_new[1].to_csv('../cleaned_data/exports.csv',index= False)
features_new[2].to_csv('../cleaned_data/imports.csv',index= False)
features_new[3].to_csv('../cleaned_data/population.csv',index= False)
features_new[4].to_csv('../cleaned_data/generation_total.csv',index= False)
features_new[5].to_csv('../cleaned_data/capacity_total.csv',index= False)
features_new[6].to_csv('../cleaned_data/generation_renewable.csv',index= False)
features_new[7].to_csv('../cleaned_data/capacity_renewable.csv',index= False)


features_new1[0].to_csv('../cleaned_data/generation_nuclear.csv',index= False)
features_new1[1].to_csv('../cleaned_data/generation_fossil.csv',index= False)
features_new1[2].to_csv('../cleaned_data/generation_hydrodinamic.csv',index= False)
features_new1[3].to_csv('../cleaned_data/capacity_nuclear.csv',index= False)
features_new1[4].to_csv('../cleaned_data/capacity_fossil.csv',index= False)
features_new1[5].to_csv('../cleaned_data/capacity_hydrodinamic.csv',index= False)

#rankings
ranking_consumption_2017.to_csv('../cleaned_data/ranking/ranking_consumption_2017.csv',index= False)
ranking_renewable_2017.to_csv('../cleaned_data/ranking/ranking_renewable_2017.csv',index= False)
ranking_norenewable_2017.to_csv('../cleaned_data/ranking/ranking_norenewable_2017.csv',index= False)
ranking_CO2_emissions_2016.to_csv('../cleaned_data/ranking/ranking_CO2_emissions_2016.csv',index= False)